# Sustained activity in hierarchical modular neural networks: self-organized criticality and oscillations

**1. Neuron populations**

The network consists of N leaky integrate-and-fire neurons, randomly connected with a connection probability of $P_0 = 0.01$

### 0. Plot configuration

In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import scienceplots

plt.style.use(["science"])

### 1. Initializing the network

In [4]:
import nest
import nest.raster_plot
import pylab

nest.ResetKernel()
n = 4  # number of threads
nest.SetKernelStatus({"local_num_threads": n})

nest.SetKernelStatus({"print_time": True})

<frozen importlib._bootstrap>:241: RuntimeWarning: compiletime version 3.8 of module 'pynestkernel' does not match runtime version 3.1
<frozen importlib._bootstrap>:241: RuntimeWarning: builtins.type size changed, may indicate binary incompatibility. Expected 880 from C header, got 888 from PyObject



              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.3
 Built: May  5 2022 07:35:57

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



**Model parameters**

In [ ]:
synapse_model = "static_synapse"
neuron_model = "iaf_cond_exp"

neuron_params = {
    "V_m": -60.0,
    "E_L": -60.0,
    "V_th": -50.0,
    "V_reset": -60.0,
    "t_ref": 5.0,
    "E_ex": 0.0,
    "E_in": -80.0,
    "C_m": 200.0,
    "g_L": 10.0,
    "I_e": 200.0,
    "tau_syn_ex": 5.0,
    "tau_syn_in": 10.0,
}

In [20]:
V_reset = -60  # Membrane resting potential in mV
V_th = 20.0  # Spike threshold in mV
tau_ref = 5  # Refractory period, in ms
tau_m = 20.0  # Membrane time constant in ms


E_ex = 0  # Excitatory reversal potential in mV
E_inh = -80  # Inhibitory reversal potential in mV

N = 10_000
N_E = int(N * (4 / 5))  # Number of excitatory neurons
N_I = int(N * (1 / 5))  # Number of excitatory neurons
N_rec = N_E  # Number of neurons to record

g_ex = 0  # Excitatory synaptic conductance in M Ohm^-1
g_inh = 0  # Inhibitory synaptic conductance in M Ohm^-1
g_L = 1  # Leak conductance

tau_syn_ex = 5  # Rise time of the excitatory synaptic alpha function in ms
tau_syn_inh = 10  # Rise time of the inhibitory synaptic alpha function ms


P_0 = 0.01  # Connection probability

**Create Neurons**

```
V_th mV = -55 mV         # Threshold potential
V_reset mV = -60 mV      # Reset potential
t_ref ms = 2 ms          # Refractory period
g_L nS = 16.6667 nS      # Leak conductance
C_m pF = 250 pF          # Membrane capacitance
E_exc mV = 0 mV          # Excitatory reversal potential
E_inh mV = -85 mV        # Inhibitory reversal potential
E_L mV = -70 mV          # Leak reversal potential (aka resting potential)
tau_syn_exc ms = 0.2 ms  # Synaptic time constant of excitatory synapse
tau_syn_inh ms = 2 ms    # Synaptic time constant of inhibitory synapse
```

In [16]:
nest.SetDefaults(
    "iaf_cond_alpha",
    {
        "V_reset": V_reset,
        "E_L": V_reset,
        "V_th": V_th,
        "C_m": tau_m,
        "t_ref": tau_ref,
        "E_ex": E_ex,
        "E_in": E_inh,
        "g_L": g_L,
        "tau_syn_ex": tau_syn_ex,
        "tau_syn_in": tau_syn_inh,
    },
)

In [18]:
nodes = nest.Create("iaf_cond_alpha", N)
nodes_E = nodes[:N_E]
nodes_I = nodes[N_E:]

noise = nest.Create("poisson_generator", 1, {"rate": 20_000, "start": 0, "stop": 200})
nest.SetDefaults("spike_recorder", {"to_file": True})

spikes = nest.Create(
    "spike_recorder", 2, [{"label": "brunel-py-ex"}, {"label": "brunel-py-in"}]
)

spikes_E = spikes[:1]
spikes_I = spikes[1:]

In [ ]:
nest.CopyModel(
    "static_synapse_hom_w",
    "excitatory",
    {
        "weight": g_,
        "delay": delay,
    },
)
nest.Connect(
    nodes_E,
    nodes,
    {"rule": "fixed_indegree", "indegree": C_E},
    "excitatory",
)
nest.CopyModel(
    "static_synapse_hom_w",
    "inhibitory",
    {
        "weight": J_I,
        "delay": delay,
    },
)
nest.Connect(
    nodes_I,
    nodes,
    {"rule": "fixed_indegree", "indegree": C_I},
    "inhibitory",
)